# Downloads PDB Structure Information
**[Work in progress]**

This notebook downloads 3D-structure information for SARS-CoV-2 proteins

Data sources: 
[RCSB Protein Data Bank](https://www.rcsb.org), 
[PDBe Protein Data Bank Europe](https://www.ebi.ac.uk/pdbe/)

Author: Peter Rose (pwrose@ucsd.edu)

In [19]:
import os
import numpy as np
import pandas as pd
import requests
import json
import dateutil
from pathlib import Path
from py2neo import Graph
from rcsbsearch import TextQuery
from rcsbsearch import rcsb_attributes as attrs

In [20]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [21]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-328d8379-6ab4-4cc1-a397-2de37909d2e4/installation-4.1.0/import


In [22]:
taxonomy_id = '2697049'

### Find PDB Structures containing SARS-CoV-2 proteins or nucleic acids

In [23]:
query = attrs.rcsb_entity_source_organism.taxonomy_lineage.id == taxonomy_id
entries = query('entry')
df = pd.DataFrame(entries, columns=['pdbId'])

In [24]:
df.head()

,pdbId
0,5R84
1,5R83
2,5R7Y
3,5R80
4,5R82


In [25]:
print("Number of structures:", df['pdbId'].shape[0])

Number of structures: 489


### Get structure metadata

In [26]:
def get_pdb_entry_data(row):
    pdb_id = row['pdbId']
    
    response = json.loads(requests.get(f'http://data.rcsb.org/rest/v1/core/entry/{pdb_id}').text)
    
    data = np.empty(9, dtype=object)

    data[0] = pdb_id
    data[1] = response['struct']['title']
    data[2] = response['struct'].get('pdbx_descriptor', '')
    data[3] = response['rcsb_accession_info']['deposit_date']
    data[4] = response['rcsb_accession_info']['initial_release_date']
    data[5] = response['pdbx_vrpt_summary'].get('pdbresolution', '')
    data[6] = response['pdbx_vrpt_summary'].get('pdbr', '')
    data[7] = response['pdbx_vrpt_summary'].get('pdbrfree', '')      
    data[8] = response['exptl'][0]['method']
    
    return data

In [27]:
structures = df.apply(get_pdb_entry_data, axis=1, result_type='expand')

In [28]:
structures.columns = ['pdbId', 'title', 'description', 'depositDate', 'releaseDate', 'resolution', 'rFactor', 'rFree', 'method']

In [29]:
structures['depositDate'] = structures['depositDate'].apply(lambda d: dateutil.parser.parse(d[:10]))
structures['releaseDate'] = structures['releaseDate'].apply(lambda d: dateutil.parser.parse(d[:10]))

In [30]:
structures['pdbId'] = 'pdb:' + structures['pdbId']

In [31]:
structures.head()

,pdbId,title,description,depositDate,releaseDate,resolution,rFactor,rFree,method
0,pdb:5R84,PanDDA analysis group deposition -- Crystal St...,SARS-CoV-2 main protease,2020-03-03,2020-03-11,1.83,0.22,0.29,X-RAY DIFFRACTION
1,pdb:5R83,PanDDA analysis group deposition -- Crystal St...,SARS-CoV-2 main protease,2020-03-03,2020-03-11,1.58,0.18,0.21,X-RAY DIFFRACTION
2,pdb:5R7Y,PanDDA analysis group deposition -- Crystal St...,SARS-CoV-2 main protease,2020-03-03,2020-03-11,1.65,0.18,0.24,X-RAY DIFFRACTION
3,pdb:5R80,PanDDA analysis group deposition -- Crystal St...,SARS-CoV-2 main protease,2020-03-03,2020-03-11,1.93,0.17,0.23,X-RAY DIFFRACTION
4,pdb:5R82,PanDDA analysis group deposition -- Crystal St...,SARS-CoV-2 main protease,2020-03-03,2020-03-11,1.31,0.18,0.21,X-RAY DIFFRACTION


In [32]:
structures.to_csv(NEO4J_IMPORT / "01f-PDBStructure.csv", index=False)

### Get PDB Chain - UniProt sequence mappings

In [15]:
sifts_url = 'http://ftp.ebi.ac.uk/pub/databases/msd/sifts/flatfiles/tsv/uniprot_segments_observed.tsv.gz'

In [16]:
segments = pd.read_csv(sifts_url, sep='\t', skiprows=1, dtype=str)
segments.head()

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END
0,113l,A,P00720,1,162,1,162,1,162
1,105m,A,P02185,1,153,1,153,2,154
2,120l,A,P00720,1,162,1,162,1,162
3,123l,A,P00720,1,162,1,162,1,162
4,128l,A,P00720,1,162,1,162,1,162


In [17]:
segments.rename(columns={'PDB': 'pdbId', 'CHAIN': 'chainId', 'SP_PRIMARY': 'accession'}, inplace=True)
segments.rename(columns={'RES_BEG': 'seqresStart', 'RES_END': 'seqresEnd'}, inplace=True)
segments.rename(columns={'PDB_BEG': 'pdbStart', 'PDB_END': 'pdbEnd'}, inplace=True)
segments.rename(columns={'SP_BEG': 'uniprotStart', 'SP_END': 'uniprotEnd'}, inplace=True)

In [18]:
segments.head()

,pdbId,chainId,accession,seqresStart,seqresEnd,pdbStart,pdbEnd,uniprotStart,uniprotEnd
0,113l,A,P00720,1,162,1,162,1,162
1,105m,A,P02185,1,153,1,153,2,154
2,120l,A,P00720,1,162,1,162,1,162
3,123l,A,P00720,1,162,1,162,1,162
4,128l,A,P00720,1,162,1,162,1,162


In [19]:
segments['pdbId'] = segments['pdbId'].str.upper()

In [20]:
segments['pdbChainId'] = segments['pdbId'] + "." + segments['chainId']

In [21]:
coverage = segments.merge(df, on='pdbId')

### Sort segments by uniprot residue number

In [22]:
coverage['uniprotStart'] = coverage['uniprotStart'].astype(int)
coverage['uniprotEnd'] = coverage['uniprotEnd'].astype(int)
coverage['length'] = coverage['uniprotEnd'] - coverage['uniprotStart'] + 1
coverage.sort_values(by='uniprotStart', inplace=True)
coverage['uniprotStart'] = coverage['uniprotStart'].astype(str)
coverage['uniprotEnd'] = coverage['uniprotEnd'].astype(str)

### Assign CURIES

In [23]:
coverage['accession'] = 'uniprot:' + coverage['accession']
coverage['pdbId'] = 'pdb:' + coverage['pdbId']
coverage['pdbChainId'] = 'pdb:' + coverage['pdbChainId']

In [24]:
coverage.head()

,pdbId,chainId,accession,seqresStart,seqresEnd,pdbStart,pdbEnd,uniprotStart,uniprotEnd,pdbChainId,length
796,pdb:6ZM7,LH,uniprot:P32969,1,190,1,190,1,190,pdb:6ZM7.LH,190
829,pdb:6ZM7,Ln,uniprot:P62945,1,24,1,24,1,24,pdb:6ZM7.Ln,24
198,pdb:6ZME,SK,uniprot:P46783,1,98,1,98,1,98,pdb:6ZME.SK,98
178,pdb:6ZME,Ln,uniprot:P62945,1,24,1,24,1,24,pdb:6ZME.Ln,24
162,pdb:6ZME,LY,uniprot:P61254,1,134,1,134,1,134,pdb:6ZME.LY,134


### Group data by PDB chains

In [25]:
coverage = coverage.groupby(['pdbId','chainId','pdbChainId','accession']).agg(list).reset_index()

In [26]:
coverage.tail()

,pdbId,chainId,pdbChainId,accession,seqresStart,seqresEnd,pdbStart,pdbEnd,uniprotStart,uniprotEnd,length
371,pdb:7JZ0,D,pdb:7JZ0.D,uniprot:P0DTD1,[20],[134],[4271],[4385],[4271],[4385],[115]
372,pdb:7JZU,B,pdb:7JZU.B,uniprot:P0DTC2,[334],[526],[334],[526],[334],[526],[193]
373,pdb:7K1O,A,pdb:7K1O.A,uniprot:P0DTD1,[25],[369],[2],[346],[6453],[6797],[345]
374,pdb:7K1O,B,pdb:7K1O.B,uniprot:P0DTD1,[25],[369],[2],[346],[6453],[6797],[345]
375,pdb:7K1O,C,pdb:7K1O.C,uniprot:P0DTD1,[25],[369],[2],[346],[6453],[6797],[345]


### Create semicolon separated string of residue numbers so they can be represented in a csv file

In [27]:
coverage['uniprotStart'] = coverage['uniprotStart'].apply(lambda x: ';'.join(x))
coverage['uniprotEnd'] = coverage['uniprotEnd'].apply(lambda x: ';'.join(x))
coverage['seqresStart'] = coverage['seqresStart'].apply(lambda x: ';'.join(x))
coverage['seqresEnd'] = coverage['seqresEnd'].apply(lambda x: ';'.join(x))
coverage['pdbStart'] = coverage['pdbStart'].apply(lambda x: ';'.join(x))
coverage['pdbEnd'] = coverage['pdbEnd'].apply(lambda x: ';'.join(x))

In [28]:
coverage['residues'] = coverage['length'].apply(lambda x: sum(x))

In [29]:
coverage.tail()

,pdbId,chainId,pdbChainId,accession,seqresStart,seqresEnd,pdbStart,pdbEnd,uniprotStart,uniprotEnd,length,residues
371,pdb:7JZ0,D,pdb:7JZ0.D,uniprot:P0DTD1,20,134,4271,4385,4271,4385,[115],115
372,pdb:7JZU,B,pdb:7JZU.B,uniprot:P0DTC2,334,526,334,526,334,526,[193],193
373,pdb:7K1O,A,pdb:7K1O.A,uniprot:P0DTD1,25,369,2,346,6453,6797,[345],345
374,pdb:7K1O,B,pdb:7K1O.B,uniprot:P0DTD1,25,369,2,346,6453,6797,[345],345
375,pdb:7K1O,C,pdb:7K1O.C,uniprot:P0DTD1,25,369,2,346,6453,6797,[345],345


In [30]:
coverage.to_csv(NEO4J_IMPORT / "01f-PDBChain.csv", index=False)